In [1]:
#!pip install sister

In [2]:
import sister
embedder = sister.MeanEmbedding(lang="en")

Loading model...


In [3]:
import os
import pandas as pd
import json
import gzip
import fasttext
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [4]:
all_files_cleaned = pd.read_csv('/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files/all_files_cleaned.csv')

In [5]:
all_files_cleaned.prob_matches.value_counts().head(9)

text_description    50
person_name         15
number              15
date                14
place               11
rating              10
address             10
time                 9
name                 7
Name: prob_matches, dtype: int64

In [10]:
all_files_cleaned[all_files_cleaned['prob_matches'] == 'rating']

,Unnamed: 0,schema_type,column_name,data_type,percentage,filename,num_tables,prob_matches
6,6,aggregaterating/ Product_stats.csv,aggregaterating,string,0.084252,Product_stats.csv,36665.000000,rating
48,49,aggregaterating/ Recipe_stats.csv,aggregaterating,string,0.422382,Recipe_stats.csv,3695.000000,rating
60,61,aggregaterating/ LocalBusiness_stats.csv,aggregaterating,string,0.181186,LocalBusiness_stats.csv,2138.000000,rating
121,122,aggregaterating/ CreativeWork_stats.csv,aggregaterating,string,0.005901,CreativeWork_stats.csv,480.000000,rating
123,124,ratingvalue/ Product_stats.csv,ratingvalue,string,0.001062,Product_stats.csv,462.000000,rating
124,125,aggregaterating/ Hotel_stats.csv,aggregaterating,string,0.415146,Hotel_stats.csv,455.000000,rating
128,129,aggregaterating/ Restaurant_stats.csv,aggregaterating,string,0.369058,Restaurant_stats.csv,427.000000,rating
133,134,bestrating/ Product_stats.csv,bestrating,string,0.000807,Product_stats.csv,351.000000,rating
146,147,aggregaterating/ Book_stats.csv,aggregaterating,string,0.157643,Book_stats.csv,297.157643,rating
150,151,worstrating/ Product_stats.csv,worstrating,string,0.000616,Product_stats.csv,268.000000,rating


In [103]:
# read all files from prob_matches file
#path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/Restaurant/Restaurant_minimum3/cleaned'
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/Book/All'
#path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/product/product_top100/cleaned'
csv_files = [file for file in os.listdir(path) if file.endswith('.gz')]

# append all columns which have the column name -> maybe easier to just use 1 or 2 columns -> less computation time
right_tables = []
col = 'aggregaterating'
#col = 'ratingvalue'
#col = 'bestrating'
#col = 'worstrating'

# get first few files with looked for column
for file in csv_files:
    df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
    if col in df.columns:
        right_tables.append(file)
        if len(right_tables) > 10:
            break

In [104]:
right_tables

['Book_loyalbooks.com_September2020.json.gz',
 'Book_hyip.com_September2020.json.gz',
 'Book_buythiz.com_September2020.json.gz',
 'Book_jafaripub.com_September2020.json.gz',
 'Book_stephensdrivingschool.co.uk_September2020.json.gz',
 'Book_senscritique.com_September2020.json.gz',
 'Book_french-bookys.org_September2020.json.gz',
 'Book_pandora-ring.org.uk_September2020.json.gz',
 'Book_apogeonline.com_September2020.json.gz',
 'Book_romancecomics.com_September2020.json.gz',
 'Book_publicvm.com_September2020.json.gz']

In [105]:
# read tables
vals = []
for file in right_tables:
    df = pd.read_json(os.path.join(path, '{}'.format(file)), compression='gzip', lines=True)
    vals.append(df[col].to_list())

In [106]:
vals_new = []
# basic cleaning (check if needed)
for elem in vals:
    for e in elem:
        vals_new.append(re.sub("[{}:,'']+", '', str(e)))

In [107]:
# get mean embedding vector for each row

#vector_product_aggregaterating = embedder(' '.join(word[0] for word in vals_new))
#vector_product_ratingvalue = embedder(' '.join(word[0] for word in vals_new))
#vector_product_bestrating = embedder(' '.join(word[0] for word in vals_new))
#vector_product_worstrating = embedder(' '.join(word[0] for word in vals_new))
#vector_recipe_aggregaterating = embedder(' '.join(word[0] for word in vals_new))
#vector_localbusiness_aggregaterating = embedder(' '.join(word[0] for word in vals_new))
#vector_creativework_aggregaterating = embedder(' '.join(word[0] for word in vals_new))
#vector_hotel_aggregaterating = embedder(' '.join(word[0] for word in vals_new))
#vector_restaurant_aggregaterating = embedder(' '.join(word[0] for word in vals_new))
vector_book_aggregaterating = embedder(' '.join(word[0] for word in vals_new))


In [116]:
# cosine similarity
from scipy import spatial
cosine_similarity = 1 - spatial.distance.cosine(vector_hotel_aggregaterating, vector_book_aggregaterating)
print(cosine_similarity)

0.967116117477417
